In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
import re
import dateparser 

In [5]:
# here we scrape information on president's term and party
# 
# NOTE: Britiania seems to be attempting to block web scrapers. When this happened, a regular requests
# approach will fail. To bypass this, you will need to use  selenium. 
# The following code should work if site is blocking scraper:

driver = webdriver.Firefox()

url = "https://www.britannica.com/topic/Presidents-of-the-United-States-1846696"
driver.get(url)
driver.implicitly_wait(10)

page_source = driver.page_source

presidents = pd.read_html(page_source)[0]
presidents.tail(20)


,Unnamed: 0,no.,president,birthplace,political party,term
28,NaN,29,Warren G. Harding,Ohio,Republican,1921–23*
29,NaN,30,Calvin Coolidge,Vt.,Republican,1923–29
30,NaN,31,Herbert Hoover,Iowa,Republican,1929–33
31,NaN,32,Franklin D. Roosevelt,N.Y.,Democratic,1933–45*
32,NaN,33,Harry S. Truman,Mo.,Democratic,1945–53
33,NaN,34,Dwight D. Eisenhower,Texas,Republican,1953–61
34,NaN,35,John F. Kennedy,Mass.,Democratic,1961–63*
35,NaN,36,Lyndon B. Johnson,Texas,Democratic,1963–69
36,NaN,37,Richard M. Nixon,Calif.,Republican,1969–74**
37,NaN,38,Gerald R. Ford,Neb.,Republican,1974–77


In [6]:
# note that the last two rows contains non-presidential information
# let's remove these last two rows...
presidents = presidents.drop([int(len(presidents)-1), int(len(presidents)-2)])

presidents.shape

(46, 6)

In [ ]:
# let's split term colunn into two columns: from and to
presidents['from'] = presidents['term'].apply(lambda x: dateparser.parse(x.split("–")[0]).year)

def to_year(row):    
    # remove all non-digit characters from s except -
    row['term'] = re.sub(r"[^\d-]", "", row['term'])
    term_list = row['term'].split("–")
    if  len(term_list)== 1:
        return row['from']
    elif len(term_list) == 2:
        return str(row['from'])[:2] + term_list[1]
    else:
        return "bad data"
    return row
    
presidents['to'] = presidents.apply(lambda row: to_year(row), axis=1)

presidents

,Unnamed: 0,no.,president,birthplace,political party,term,from,to
0,NaN,1,George Washington,Va.,Federalist,1789–97,1789,1789
1,NaN,2,John Adams,Mass.,Federalist,1797–1801,1797,1797
2,NaN,3,Thomas Jefferson,Va.,Democratic-Republican,1801–09,1801,1801
3,NaN,4,James Madison,Va.,Democratic-Republican,1809–17,1809,1809
4,NaN,5,James Monroe,Va.,Democratic-Republican,1817–25,1817,1817
5,NaN,6,John Quincy Adams,Mass.,National Republican,1825–29,1825,1825
6,NaN,7,Andrew Jackson,S.C.,Democratic,1829–37,1829,1829
7,NaN,8,Martin Van Buren,N.Y.,Democratic,1837–41,1837,1837
8,NaN,9,William Henry Harrison,Va.,Whig,1841*,1841,1841
9,NaN,10,John Tyler,Va.,Whig,1841–45,1841,1841


In [ ]:
presidents.to_csv("presidents.csv", encoding="utf-8",quotechar="'",index=False)

In [ ]:
# now, let's merge speeches dataframe with presidents dataframe - since there are some slight variations in the 
# president names between these two tables, we will use a fuzzy merge technique
import difflib 

df1 = pd.read_csv("presidential_speeches.csv", encoding="utf-8",quotechar="'")
df2 = pd.read_csv("presidents.csv", encoding="utf-8",quotechar="'")

# change the column name of the presidents dataframe to match the speeches dataframe
# In the presidents dataframe, the column name is 'president'. In the speeches dataframe, the column name is 'name'
df2.rename(columns={'president':'name'}, inplace=True)

#create duplicate column to retain team name from df2
df2['name_match'] = df2['name']

#convert team name in df2 to team name it most closely matches in df1
df2['name'] = df2['name'].apply(lambda x: difflib.get_close_matches(x, df1['name'])[0])

df2

,Unnamed: 0,no.,name,birthplace,political party,term,from,to,name_match
0,NaN,1,George Washington,Va.,Federalist,1789–97,1789,1789,George Washington
1,NaN,2,John Adams,Mass.,Federalist,1797–1801,1797,1797,John Adams
2,NaN,3,Thomas Jefferson,Va.,Democratic-Republican,1801–09,1801,1801,Thomas Jefferson
3,NaN,4,James Madison,Va.,Democratic-Republican,1809–17,1809,1809,James Madison
4,NaN,5,James Monroe,Va.,Democratic-Republican,1817–25,1817,1817,James Monroe
5,NaN,6,John Quincy Adams,Mass.,National Republican,1825–29,1825,1825,John Quincy Adams
6,NaN,7,Andrew Jackson,S.C.,Democratic,1829–37,1829,1829,Andrew Jackson
7,NaN,8,Martin Van Buren,N.Y.,Democratic,1837–41,1837,1837,Martin Van Buren
8,NaN,9,William Harrison,Va.,Whig,1841*,1841,1841,William Henry Harrison
9,NaN,10,John Tyler,Va.,Whig,1841–45,1841,1841,John Tyler


In [ ]:
#merge the DataFrames into one
df3 = df1.merge(df2)

#view final DataFrame
df3

,name,title,date,info,speech,Unnamed: 0,no.,birthplace,political party,term,from,to,name_match
0,Joe Biden,"February 21, 2023: Remarks on the One-Year Ann...",2023-02-21,"Speaking at the Royal Castle in Warsaw, Poland...","THE PRESIDENT: Hello, Poland! One of our g...",NaN,46,Pa.,Democratic,2021–,2021,2021,Joe Biden
1,Joe Biden,"February 7, 2023: State of the Union Address",2023-02-07,"In his State of the Union Address, President J...",Mr. Speaker. Madam Vice President. Our F...,NaN,46,Pa.,Democratic,2021–,2021,2021,Joe Biden
2,Joe Biden,"September 21, 2022: Speech before the 77th Ses...",2022-09-21,President Joe Biden addresses the 77th session...,"Thank you. Mr. President, Mr. Secretary...",NaN,46,Pa.,Democratic,2021–,2021,2021,Joe Biden
3,Joe Biden,"September 1, 2022: Remarks on the Continued Ba...",2022-09-01,"President Joe Biden speaks in Philadelphia, Pe...","THE PRESIDENT: My fellow Americans, please,...",NaN,46,Pa.,Democratic,2021–,2021,2021,Joe Biden
4,Joe Biden,"May 24, 2022: Remarks on School Shooting in Uv...",2022-05-24,President Biden makes an impassioned plea to s...,"Good evening, fellow Americans. I had hope...",NaN,46,Pa.,Democratic,2021–,2021,2021,Joe Biden
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086,George Washington,"December 29, 1790: Talk to the Chiefs and Coun...",1790-12-29,The President reassures the Seneca Nation that...,"I the President of the United States, by my ...",NaN,1,Va.,Federalist,1789–97,1789,1789,George Washington
1087,George Washington,"December 8, 1790: Second Annual Message to Con...",1790-12-08,Washington focuses on commerce in his second a...,Fellow citizens of the Senate and House of R...,NaN,1,Va.,Federalist,1789–97,1789,1789,George Washington
1088,George Washington,"January 8, 1790: First Annual Message to Congress",1790-01-08,"In a wide-ranging speech, President Washington...",Fellow Citizens of the Senate and House of Re...,NaN,1,Va.,Federalist,1789–97,1789,1789,George Washington
1089,George Washington,"October 3, 1789: Thanksgiving Proclamation",1789-10-03,"At the request of Congress, Washington establi...",Whereas it is the duty of all Nations to ackn...,NaN,1,Va.,Federalist,1789–97,1789,1789,George Washington


In [ ]:
df3.to_csv("presidential_speeches.csv", encoding="utf-8",quotechar="'",index=False)

In [ ]:
df3['political party'].value_counts()

political party
Democratic               497
Republican               439
Democratic-Republican     56
Democratic (Union)        31
Whig                      30
Federalist                30
National Republican        8
Name: count, dtype: int64